In [ ]:
try:
    import tsai
except:
    !pip install tsai 

<hr style="border: solid 3px blue;">

# Introduction

![](https://media3.giphy.com/media/gSl7e703puA6tSSfqW/giphy.gif)

Picture Credit:https://media3.giphy.com

Now there is a wall in front of us. There are several ways to get over a wall. You can barely jump over it, you can jump over it, or you can break the wall and move on.

There are also several ways to solve the problem. We will try to solve this problem in several ways. Let's see how to solve this problem in various ways.

One more thing before I start, I'm not going to do as much preprocessing as possible in this notebook. In fact, one of the advantages of deep learning is that it does not require much preprocessing. So, let's give up the temptation to get high scores and make the most of the advantages of deep learning.

## Checking Files
> * train.csv - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
> 1. **sequence** - a unique id for each sequence
> 2. **subject** - a unique id for the subject in the experiment
> 3. **step** - time step of the recording, in one second intervals
> 4. **sensor_00 - sensor_12**- the value for each of the thirteen sensors at that time step
> * train_labels.csv - the class label for each sequence.
> 1. **sequence** - the unique id for each sequence.
> 2. **state** - the state associated to each sequence. This is the target which you are trying to predict.
> * test.csv - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.
> * sample_submission.csv - a sample submission file in the correct format

## Checing Metrics

![](https://miro.medium.com/max/722/1*pk05QGzoWhCgRiiFbz-oKQ.png)

Picture Credit: https://miro.medium.com

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

**What are ROC and AUROC**
> The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. The true-positive rate is also known as sensitivity, recall or probability of detection. The false-positive rate is also known as probability of false alarm and can be calculated as (1 − specificity). It can also be thought of as a plot of the power as a function of the Type I Error of the decision rule (when the performance is calculated from just a sample of the population, it can be thought of as estimators of these quantities). The ROC curve is thus the sensitivity or recall as a function of fall-out. 

Ref: https://en.wikipedia.org/wiki/Receiver_operating_characteristic

-------------------------------------
# Setting Up

In [ ]:
import numpy as np 
import pandas as pd 
from fastai.text.all import *

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

import warnings
warnings.filterwarnings(action='ignore')

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

In [ ]:
from tsai.all import *
computer_setup()

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
submission_df = pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")
labels_df = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")

----------------------------------------
# Checking Target Imbalance

In [ ]:
labels_df.head().T.style.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'white'})

In [ ]:
colors = ['gold', 'mediumturquoise']
labels = ['0','1']
values = labels_df['state'].value_counts()/labels_df['state'].shape[0]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(
    title_text="Target Balance",
    title_font_color="white",
    legend_title_font_color="yellow",
    paper_bgcolor="black",
    plot_bgcolor='black',
    font_color="white",
)
fig.show()

<span style="color:Blue"> Observation:
* OK! Target is well-balanced.

------------------------------------
# Preprocessing

In [ ]:
train.head().T.style.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'white'})

## Augmentation

In [ ]:
features = train.columns.tolist()[3:]
def augument(df):
    for feature in features:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1)
        df.fillna(0, inplace=True)
        df[feature + '_diff1'] = df[feature] - df[feature + '_lag1'] 
        df[feature + '_flip'] = df[feature] * -1.

augument(train)
augument(test)

## Scaling

In [ ]:
features = train.columns.tolist()
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.transform(test[features])

Since it is a recording of the sensors for 60 seconds, we set the window to 60.

In [ ]:
Window = 60

Let's change it to a 3D ndarray as shown below.
* Samples
* Variables
* Length (aka time or sequence steps)

In [ ]:
y_train = labels_df['state'].to_numpy()
train = train.drop(["sequence", "subject", "step"], axis=1).to_numpy()
train = train.reshape(-1, Window, train.shape[-1])
train = np.transpose(train, (0, 2, 1))

test = test.drop(["sequence", "subject", "step"], axis=1).to_numpy()
test = test.reshape(-1, Window, test.shape[-1])
test = np.transpose(test, (0, 2, 1))

In [ ]:
train.shape, y_train.shape

----------------------------------------
# Splitting Train/Valid Dataset

We make the valid dateset randomly with a size of 30% as shown below.

In [ ]:
splits = get_splits(y_train, valid_size=.3, stratify=True, random_state=23, shuffle=True)
splits

-----------------------------------------------
# Making Pipeline and Dataloaders

In [ ]:
dls = TSDataLoaders.from_numpy(train, y_train, bs=32, val_bs=64, splits=splits, num_workers=0)

---------------------------------
# Checking Batch

In [ ]:
dls.show_batch(sharey=True)

------------------------------
# Modeling
![](https://miro.medium.com/max/700/1*QCeZup5dSkAtd3JvJeDa2Q.png)

Ref: https://miro.medium.com

**What is InceptionTime?**
> InceptionTime was primarily inspired by CNNs for computer vision problems, and we, therefore, expect our model to learn features in a similar fashion. For example, in image classification, the neurons in the bottom layers learn to identify low-level (local) features such as lines, while the neurons in higher layers learn to detect high-level (global) features such as shapes (e.g. eyes). Similarly, we expect the bottom-layer neurons of InceptionTime to capture the local structure of a time series such as lines and curves, and the top-layer neurons to identify various shape patterns such as “valleys” and “hills”.

Ref: https://towardsdatascience.com/deep-learning-for-time-series-classification-inceptiontime-245703f422db

CNN itself is widely used in the field of 'Image classification', and 1D CNN is suitable for analyzing NLP (Natural Language Processing) or sensor & signal data. It is also good for identifying simple patterns within data and works well for time series analysis.

Here, we will use the InceptionTime model using the inception block using 1D-CNN.

In [ ]:
%%time
model = InceptionTimePlus(train.shape[-2], dls.c)
learn = Learner(dls,
                model,
                metrics=[accuracy,RocAucBinary()],
                cbs = [EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=2)]
               )

![](https://miro.medium.com/max/770/1*Yi4LbmlBTc8JiTdjsNtfXA.png)

Picture Credit: https://miro.medium.com

In [ ]:
learn.model

------------------------------
# Training

In [ ]:
%%time
with plt.rc_context({"figure.figsize": (4,4), "figure.dpi": (200)}):
    sr = learn.lr_find()
sr.valley

In [ ]:
%%time
learn.fit_one_cycle(100, sr.valley)

In [ ]:
with plt.rc_context({"figure.figsize": (4,4), "figure.dpi": (200)}):
    learn.recorder.plot_loss()

<span style="color:Blue"> Observation:
* It seems to have been early stopping at an appropriate time.

In [ ]:
learn.plot_confusion_matrix(figsize=(3,3),dpi=200)

In [ ]:
learn.show_probas(figsize=(5,5),dpi=300)

<span style="color:Blue"> Observation:
* Looking at the above figures, it can be seen that classification is better when the Actual value is 1.

----------------------------------------
# Predicting

In [ ]:
test_dl = learn.dls.test_dl(test)
test_dl.show_batch()

In [ ]:
preds,y = learn.get_preds(dl=test_dl)
results = preds[:,1].tolist()

In [ ]:
submission_df['state'] = results

In [ ]:
submission_df.to_csv('submission.csv', index = False)

<hr style="border: solid 3px blue;">